# Naive Bayes

Naive Bayes se refiere al uso del teorema de Bayes con suposiciones ingenuas sobre la independencia entre variables.

Los clasificadores de Naive Bayes son populares en la categorización de texto. El objetivo de este modulo es aprender lo básico de Naive Bayes. Este módulo está dividido en las siguientes secciones:

+ Preparación de los datos para el análisis
+ Introducción a Naive Bayes
+ Tokenización
+ Vectorización
+ Predicción
+ Evaluación

Este módulo coincide con nuestra clase de procesamiento de lenguaje natural con Naive Bayes.

Sin embargo, antes de ir a la sección uno sobre preparación de datos, comenzaremos por una revisión de los datos que estamos usando. Para la comprención de la metodología, nosotros:

1. Primero, bajamos data de Twitter sobre microfinanza usando tweepy, una librería API de Twitter.
2. Luego, clasificamos los comentarios de acuerdo a polaridad para asi obtener data de entrenamiento para nuestro modelo.
    + Normalmente este proceso requriría clasificar manualmente ~20% de nuestros datos.
    + En nuestro ejercisio, usamos un simple algoritmo llamado TextBlob para etiquetar todo el dataset.
3. Después revisamos y corregimos manualmente las clasificaciones. 
    + (Nota, si estas interesada en ver el código usado para implementar esto, ve el archivo "twitter_web_scraping.")

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
import re
import itertools
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
!pip install wordcloud
from wordcloud import WordCloud

import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
path = '../data/'
filename = 'microfinance_tweets.csv'
try:
    data = pd.read_csv(path+filename, encoding='ISO-8859-1')
except FileNotFoundError:
    import os
    os.system(f'!git clone https://github.com/DeltaAnalytics/machine_learning_for_good_data {path}')
    data = pd.read_csv("machine_learning_for_good_data/microfinance_tweets.csv", encoding="ISO-8859-1")

In [ ]:
#revisa los datos para ver que todo salió bien
data.head(10)

La data de Twitter contiene 6 columnas:
+ Comentarios: El post de Tweet que mencionó microfinanzas
+ Fecha: La fecha del Tweet
+ Favoritos: Número de favoritos
+ Usuario: Nombre del autor del tweet
+ Polaridad: la polaridad, que representa la positividad o negatividad del comentario de acuerdo a TextBlob
+ Sentimiento: conversión de polaridad a positivo, negativo y neutro.

Notar que este ejercisio simula realisticamente lo que pasaría en el mundo real. Nuestra data es limitada, y el proceso para obtener data de entrenamiento y test de calidad es dificil. Para entender nuestra data, haremos unos pocos ejercisios abajo. 

In [ ]:
## Cual es la distribución de sentimiento?
sns.countplot(x = 'Sentiment', data = data)
data['Sentiment'].value_counts()

Parece que la mayoriá tiene comentarios positivos o neutros sobre microfinanza. De hecho, el volumen de comentarios negativos es lo más interesante y requiere mayor investigación.

In [ ]:
## Que són estos comentarios negativos?
#formatea las columnas para que los comentarios no se corten
pd.set_option('display.max_columns', None) 
pd.set_option('max_colwidth', 800)

#muestra comentarios negativos
pd.DataFrame(data.loc[data['Sentiment'] == 'negative']['Comments'].unique()[0:10])

Pareciera que alguno de los comentarios negativos no son válidos y debiesen ser clasificados como neutrales. Sin embargo, alguno de los comentarios negativos son válidos, e.g., notificación de página falsa y actividad ilegal.

Aunque hay mucho más que podríamos investigar sobre los datos, continuaremos con la preparación de los datos para el modelo.

# Prepara los datos para el modelo

El modo mas común de aplicar algoritmos de machine learning, como Naive Bayes, a anáisis de texto es convertir nuestro texto en características numéricas que el algorítmo puede entender.

La representación que crearemos es la **Bolsa de palabras**. Para pasar de nuestros datos, que actualmente están estructurados como una serie de strings, a una Bolsa de Palabras, **vectorizaremos** nuestros tweets.

Esta **vectorización** de nuestros documentos de texto requerirá unos pocos pasos claves, que describiremos en más detalle en el [Módulo 8](https://docs.google.com/presentation/d/1vaxDuUROgaqix9Mfkyb4_h0cGkGwN7AUeLU1ozhtJW8/edit#slide=id.g227403103b_0_1171):

Paso 0. Divide la data en sets de entrenamiento y test (esto no es parte de la vectorización) <br>
Paso 1. Limpia y **tokeniza** el texto <br>
Paso 2. Cuenta el número de palabras en cada texto

#### Paso 0: Crea datos de entrenamiento y test
Este paso debiese venir de forma natural. 
Hemos clasificado sentimientos para todos nuestros datos, y escogemos una division 80-20 de entrenamiento-test, con observaciones asignadas al azar.

In [ ]:
# A) Crea un set de test (20% de los datos) y entrenamiento (80% de los datos) 
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Verifica el largo de los set de test y entrenamiento para confirmar que los datos se dividieron en la proporción adecuada
print(len(train), len(test))
print("Split of {:.2%} train, {:.2%} test".format(len(train)/len(data), len(test)/len(data)))

#### Paso 1: Limpia y tokeniza el texto

Como mencionado, el modo más común de extrar vectores numéricos del texto es tener un set de **tokens** para los cuales tenemos conteos.

**Tokenización** significa convertir un documento/sentencia/tweet en *tokens* individuales, que usualmente son palabras, o unidades 'como' palabras.

Cuando creamos estos tokens 'como' palabras, queremos estandarizarlos de modo que palabras similares estén agrupadas. Esto hace los tokens más útiles que palabras sin procesar. Mira el [NLP Modulo](https://docs.google.com/presentation/d/1vaxDuUROgaqix9Mfkyb4_h0cGkGwN7AUeLU1ozhtJW8/edit#slide=id.g227403103b_0_1050) para más detalles!

Creamos una función para lemalizar** estas palabras. Afortunádamente, NLTK tiene diferentes librerias que simplifican esta tarea.

Aquí usamos el tokenizador de NLTK para lidiar con un tweet con hashtags y smileys.

In [ ]:
## From http://www.nltk.org/api/nltk.tokenize.html
tweet_tokenizer = TweetTokenizer()
example_tweet = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"
tweet_tokenizer.tokenize(example_tweet)

Nota: El tokenizador de tweet tiene la capacidad adicional de reducir el largo de algunas palabras laaaargas.

#### Paso 2) Cueta el número de palabras en cada 'documento'

Este es el paso que trae nuestro texto a la representación *Bolsa de Palabras*.

Una bolsa de palabras nos permite trabajar con text libre de forma estructurada. Usamos un simple conteo para nuestra bolsa de palabras, pero las librerias TF-IDF sklearn libraries también están disponibles y pueden ser utilizadas en otros problemas de NLP.

Usamos la función **CountVectorization** de la libreria sklearn
The TF-IDF Vectorizer is available by using:

`from sklearn.feature_extraction.text import TfidfVectorizer`

Notar la diferencia en el uso de las funciones para ajustar la bolsa de palabras de test vs la de entrenamiento.

In [ ]:
# C) Representa el texto en una bolsa de palabras por medio de CountVectorization en la libreria sklearn.
vectorizer = CountVectorizer(tokenizer=tweet_tokenizer.tokenize)
train_features = vectorizer.fit_transform(train['Comments'])
test_features =  vectorizer.transform(test['Comments'])

In [ ]:
vectorizer.get_feature_names()[0:20]

Estas son nuestras *características*, i.e., todos los tokens limpios de todos nuestros tweets. Un tweet de entrenamiento o de test puede ser representado ahora como un conteo de cuantas veces esta característica aparece. 

In [ ]:
# Prueba el vectorizador mirando el tamaño de los vectores y extractos
print('Length of Vectorizer Vocabulary: ', len(vectorizer.vocabulary_))
print('Shape of Sparse Matrix: ', test_features.shape)
print('Amount of Non-Zero occurrences: ', test_features.nnz)

# Porcentajes de valores distintos de zero
density = 100.0 * (test_features.nnz / (test_features.shape[0] * test_features.shape[1]))
print('Density: {}'.format((density)))

Tenemos 6184 *tokens* en nuestro cuerpo. La matriz de características del set de test es una representación de cada unos de los 649 tweets como un vector del conteo de tokens.

In [ ]:
# Para entender como el vectorizador guarda palabras, pondremos como input un ejemplo del test
sample_test = test['Comments'].iloc[11]
print("Sample comment: ", sample_test, "\n")
sample_vector = vectorizer.transform([sample_test])
print("Vectorization:")
print(sample_vector)

In [ ]:
# Esto significa que tenemos un conteo de palabras para palabras en estos índices.
# Podemos usar get_feature_names() en el vectorizador para ver cuales son estas palabras
print(list(vectorizer.get_feature_names()[i] for i in sample_vector.indices))

Miremos a una nube de palabras de nuestros datos para ver que palabras son las más comunes!

In [ ]:
#cloud = WordCloud(width=1600, height=1200).generate(" ".join(data['Comments'].astype(str)))
cloud = WordCloud(width=1600, height=1200).generate(" ".join(vectorizer.get_feature_names()))
plt.figure(figsize=(20, 15))
plt.imshow(cloud)
plt.axis('off')

Estas palabras tienen sentido dado nuestro foco. Tenemos también algunas urls (https, co).

# Introducción a Naive Bayes
Comenzaremos corriendo Naive Bayes en el texto y verificando que tan bien predice las clasificaciones. Este paso es muy simple ya que ya hemos preparado nuestra data en el formato de bolsa de palabras. <br>

In [ ]:
#entrena tu data usando multinomial NB de la librería sklearn
nb = MultinomialNB()
nb.fit(train_features, train['Sentiment'])

In [ ]:
#prueba con set de prueba
preds = nb.predict(test_features)

#imprime la precisión de tu modelo
accuracy = (preds == test['Sentiment'])
'Accuracy : {:.2%}'.format(accuracy.sum() / len(accuracy))

La precisión es alta ~86%. La mayoría de los análisis de sentimiento tienen una precisión de ~80% cuando has ajustado el modelo en un test de entrenamiento real. En este caso, la precisión de 86% está probáblemente inflada porque hemos usado TextBlob para determinar el sentimiento correcto en nuestro set de entrenamiento. 

Nuestra medida actual de precisión es un valor uni-dimensional. No sabemos en que nos estamos equivocando cuando clasificamos los sentimientos mal. Una herramienta para entender mejor como estamos clasificando es la matriz de confución (primero introducida en la clase de regreción logística.)

In [ ]:
## Se puede tomar como dada de la documentación de la matriz de confución en skalearn
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    Esta función imprime y grafica la matriz de confución.
    Nomalización se aplica al configurar `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
class_names = set(data['Sentiment'])
cnf_matrix = confusion_matrix(test['Sentiment'], preds)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, basic NB')

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

Los resultados de arriba muestran que estamos comunmente prediciendo comentarios positivos como neutrales. Esta mal clasificación no es tan preocupante como nuestra segunda mal clasificación mas común de clasificar comentarios negativos como positivos.

# Conclusión

Hemos revisado lo básico de Naive Bayes e investigado como mejoras pueden ser usadas en el caso de análisis de sentimiento. 

Hay muchos tipos de pequeñas mejoras que puedes aplicar a un modelo de Naive Bayes, y sualmente depende del tipo de dato. Por ejemplo, si sabes que el texto a usar es particularmente informal, sería bueno adaptar tu función analizadora para lidiar con estas informalidades. Si el texto no es informal, este cambio no cambiará mucho la precisión de tu modelo. Para más detalles de como mejoras pueden ser aplicadas a Naive bayes, mira el laboratorio "naive_bayes_detail"

Sources: https://medium.com/tensorist/classifying-yelp-reviews-using-nltk-and-scikit-learn-c58e71e962d9, https://www.dataquest.io/blog/naive-bayes-tutorial/, http://nlpforhackers.io/sentiment-analysis-intro/, http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html <br>